# extract NER Training Samples from TEI-Files fetched from a GitHub Repo

In [1]:
from github import Github
import json
import os
import acdh_tei_pyutils
from acdh_tei_pyutils.tei import TeiReader

In [2]:
clean_markup = os.path.join(acdh_tei_pyutils.__path__[0], 'files', 'clean_markup.xsl')

In [3]:
g = Github()

In [4]:
repo = g.get_repo('bleierr/NERDPool')

In [5]:
contents = repo.get_contents("RTA_1576")

In [6]:
with open("sample.jsonl", "w") as a_file:
    for x in contents:
        dl_url = x._rawData.get('download_url')
        print(dl_url)
        doc = TeiReader(xml=dl_url, xsl=clean_markup)
        parent_node = doc.any_xpath('.//tei:body')[0]
        ne_list = doc.extract_ne_offsets(
            parent_nodes='.//tei:body//tei:p',
            ne_xpath=".//*[contains(name(), 'Name') or name()='date' or name()='time']"
        )
        for y in ne_list:
            item = {
                "text": y[0],
                "entities": y[1]['entities']
            }
            a_file.write("\n")
            a_file.write(json.dumps(item, ensure_ascii=False))

https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/FR-Protokoll-Baden.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/FR-Protokoll-Stra%C3%9Fburg.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/FR-Protokoll-%C3%96sterreich-Juli.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/Geheimer_Rat-Protokoll-Wien-HHStA.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/HHStA%20Wien_AUR%201576_Oktober%2012_V2.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/HStA_Dresden_Loc10199_4_fol11_1576-05-21.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/HStA_Dresden_Loc10199_4_fol13_1576-05-25.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/HStA_Dresden_Loc10199_4_fol159_1576-06-30.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA_1576/HStA_Dresden_Loc10199_4_fol16_1576-05-23.xml
https://raw.githubusercontent.com/bleierr/NERDPool/main/RTA

In [7]:
import warnings
import spacy
from spacy.tokens import DocBin

In [8]:
nlp = spacy.blank('de')
db_train = DocBin()
db_eval = DocBin()
fails = 0
train_eval = 0
train_sample_count = 0
eval_sample_count = 0
with open('sample.jsonl') as f:
    for line in f:
        if train_eval == 5:
            train_eval = 0
        try:
            data = json.loads(line.strip())
        except:
            continue
        text = data['text']
        entities = data['entities']
        doc = nlp.make_doc(text)
        ents = []
        train_eval += 1
        for start, end, label in entities:
            span = doc.char_span(start, end, label=label)
            if span is None:
                # msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                # warnings.warn(msg)
                fails += 1
            else:
                ents.append(span)
        try:
            doc.ents = ents
            if train_eval != 4:
                db_train.add(doc)
                train_sample_count += 1
            else:
                db_eval.add(doc)
                eval_sample_count += 1
        except ValueError:
            continue
    db_train.to_disk('train.spacy')
    db_eval.to_disk('dev.spacy')
print(
    f"fails: {fails}; train: {train_sample_count}; eval: {eval_sample_count}"
)
        

fails: 192; train: 8329; eval: 2083


In [ ]:
! python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

✔ Created output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-03-17 13:47:08,872] [INFO] Set up nlp object from config
[2021-03-17 13:47:08,896] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-03-17 13:47:08,913] [INFO] Created vocabulary
[2021-03-17 13:47:08,913] [INFO] Finished initializing nlp object
[2021-03-17 13:47:18,934] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     43.56    0.00    0.00    0.00    0.00
  0     200         64.10   1435.65   38.39   87.85   24.56    0.38
  0     400        322.28    744.01   61.64   73.74   52.95    0.62
  0     600       2881.85    637.64   68.32  